In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
from pandasql import sqldf

#### 자료 읽기

In [34]:
df = pd.read_csv('./세종도시교통공사_공공자전거(어울링) 이용 현황_202204~202207_원본데이터.csvv', encoding='euc-kr')
df_spot = pd.read_csv('./세종도시교통공사_공공자전거(어울링)_대여소현황_20220830.csvv', encoding='euc-kr')

In [35]:
len(df)

1018929

In [36]:
len(df_spot)

618

In [37]:
df

,순번,자전거고유번호,시작 대여소,대여시간,반납 대여소,반납시간,주행거리,주행시간,환승 유무
0,3672508,001_0110_02460,SJ_00439,2022-04-01 00:00:52,SJ_00222,2022-04-01 00:07:31,954,6,N
1,3672509,001_0110_02587,SJ_00165,2022-04-01 00:01:17,SJ_00152,2022-04-01 00:11:25,1366,10,N
2,3672510,001_0110_01483,SJ_00273,2022-04-01 00:01:39,SJ_00581,2022-04-01 01:25:33,14720,83,N
3,3672511,001_0110_02910,SJ_00347,2022-04-01 00:01:58,SJ_00326,2022-04-01 00:11:25,1845,9,N
4,3672512,001_0110_02059,SJ_00426,2022-04-01 00:02:03,SJ_00470,2022-04-01 00:12:26,1690,10,N
...,...,...,...,...,...,...,...,...,...
1018924,4696249,001_0110_00901,SJ_00316,2022-07-31 23:56:57,SJ_00422,2022-08-01 00:02:39,648,5,N
1018925,4696250,001_0110_00949,SJ_00240,2022-07-31 23:58:36,SJ_00347,2022-08-01 00:09:13,958,10,N
1018926,4696251,001_0110_02458,SJ_00367,2022-07-31 23:59:09,SJ_00191,2022-08-01 00:05:40,1046,6,N
1018927,4696252,001_0110_02154,SJ_00390,2022-07-31 23:59:25,SJ_00237,2022-08-01 00:11:53,1340,12,N


In [38]:
df_spot

,대여소 번호,대여소 아이디,대여소 이름,사용유무,주소(구),주소(동),상세주소,위도,경도,권역,지오펜스 거리,등록일,수정일
0,1,SJ_00001,보람동_세종시청 정문1,Y,세종,보람동,세종특별자치시 보람동 702,127.289038,36.478950,3,30,2018-06-19 19:37:25,2021-12-27 15:34:26
1,2,SJ_00002,보람동_시드니하트,Y,세종,보람동,세종특별자치시 보람동 616-1,127.294110,36.484045,3,30,2018-07-09 20:33:41,2021-04-20 13:56:15
2,3,SJ_00003,보람동_세종시청 후문,Y,세종,보람동,세종특별자치시 보람동 663,127.288405,36.480258,3,30,2018-06-21 15:44:32,2020-12-10 10:24:00
3,4,SJ_00004,보람동_세종시청후문 잔디광장,Y,세종,보람동,세종특별자치시 보람동 626-2,127.289090,36.480842,3,30,2018-06-21 15:49:04,2020-12-10 10:24:00
4,5,SJ_00005,보람동_호려울302동 건너편,Y,세종,보람동,세종특별자치시 보람동 664-47,127.281104,36.477518,3,30,2018-06-21 15:51:54,2020-12-10 10:24:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,661,SJ_00661,어진동_한뜰401동 앞,Y,세종,어진동,세종특별자치시 어진동 520,127.255611,36.501082,1,30,2022-05-31 14:26:39,2022-07-22 14:33:06
614,662,SJ_00662,집현동_새나루마을 803동 로터리,Y,세종,집현동,세종특별자치시 집현동 785-94,127.323787,36.489735,4,25,2022-07-29 14:11:12,2022-07-29 18:59:20
615,900,SM_00001,삼미,N,세종,가산동,서울특별시 금천구 가산동,126.880623,37.484362,1,200,2018-10-12 12:51:30,2018-10-12 12:52:23
616,990,DJ_00001,두드림,N,세종,장대동,대전 유성구 장대동 336-3,127.337299,36.363496,0,70,2018-06-28 13:25:49,2020-10-14 13:12:06


### 데이터 전처리
#### 이용거리 구간

In [39]:
m_ranges = [-1, 200, 500, 700, 1000, 1500, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 15000, 20000, 30000, 100000, 35000000]
m_labels = ['0.2KM', '0.5KM', '0.7KM', '1KM', '1.5KM','2KM','3KM','4KM','5KM','6KM','7KM','8KM','9KM','10KM','15KM','20KM','30KM','100KM', '100KM~']
df['mile_bin'] = pd.cut(df['주행거리'], m_ranges, labels=m_labels)
df

,순번,자전거고유번호,시작 대여소,대여시간,반납 대여소,반납시간,주행거리,주행시간,환승 유무,mile_bin
0,3672508,001_0110_02460,SJ_00439,2022-04-01 00:00:52,SJ_00222,2022-04-01 00:07:31,954,6,N,1KM
1,3672509,001_0110_02587,SJ_00165,2022-04-01 00:01:17,SJ_00152,2022-04-01 00:11:25,1366,10,N,1.5KM
2,3672510,001_0110_01483,SJ_00273,2022-04-01 00:01:39,SJ_00581,2022-04-01 01:25:33,14720,83,N,15KM
3,3672511,001_0110_02910,SJ_00347,2022-04-01 00:01:58,SJ_00326,2022-04-01 00:11:25,1845,9,N,2KM
4,3672512,001_0110_02059,SJ_00426,2022-04-01 00:02:03,SJ_00470,2022-04-01 00:12:26,1690,10,N,2KM
...,...,...,...,...,...,...,...,...,...,...
1018924,4696249,001_0110_00901,SJ_00316,2022-07-31 23:56:57,SJ_00422,2022-08-01 00:02:39,648,5,N,0.7KM
1018925,4696250,001_0110_00949,SJ_00240,2022-07-31 23:58:36,SJ_00347,2022-08-01 00:09:13,958,10,N,1KM
1018926,4696251,001_0110_02458,SJ_00367,2022-07-31 23:59:09,SJ_00191,2022-08-01 00:05:40,1046,6,N,1.5KM
1018927,4696252,001_0110_02154,SJ_00390,2022-07-31 23:59:25,SJ_00237,2022-08-01 00:11:53,1340,12,N,1.5KM


#### 이용시간 구간

In [40]:
df['주행시간'].max()

3119

In [41]:
h_ranges = [-1,15, 30, 45, 60, 90, 120, 150, 180, 240, 300, 360, 420, 3200]
h_labels = ['15분', '30분', '45분', '60분', '90분','120분','150분','180분','240분','300분','360분','420분','7시간~']
df['hour_bin'] = pd.cut(df['주행시간'], h_ranges, labels = h_labels)
df

,순번,자전거고유번호,시작 대여소,대여시간,반납 대여소,반납시간,주행거리,주행시간,환승 유무,mile_bin,hour_bin
0,3672508,001_0110_02460,SJ_00439,2022-04-01 00:00:52,SJ_00222,2022-04-01 00:07:31,954,6,N,1KM,15분
1,3672509,001_0110_02587,SJ_00165,2022-04-01 00:01:17,SJ_00152,2022-04-01 00:11:25,1366,10,N,1.5KM,15분
2,3672510,001_0110_01483,SJ_00273,2022-04-01 00:01:39,SJ_00581,2022-04-01 01:25:33,14720,83,N,15KM,90분
3,3672511,001_0110_02910,SJ_00347,2022-04-01 00:01:58,SJ_00326,2022-04-01 00:11:25,1845,9,N,2KM,15분
4,3672512,001_0110_02059,SJ_00426,2022-04-01 00:02:03,SJ_00470,2022-04-01 00:12:26,1690,10,N,2KM,15분
...,...,...,...,...,...,...,...,...,...,...,...
1018924,4696249,001_0110_00901,SJ_00316,2022-07-31 23:56:57,SJ_00422,2022-08-01 00:02:39,648,5,N,0.7KM,15분
1018925,4696250,001_0110_00949,SJ_00240,2022-07-31 23:58:36,SJ_00347,2022-08-01 00:09:13,958,10,N,1KM,15분
1018926,4696251,001_0110_02458,SJ_00367,2022-07-31 23:59:09,SJ_00191,2022-08-01 00:05:40,1046,6,N,1.5KM,15분
1018927,4696252,001_0110_02154,SJ_00390,2022-07-31 23:59:25,SJ_00237,2022-08-01 00:11:53,1340,12,N,1.5KM,15분


#### 이용요일, 시간대

In [42]:
days = ['월요일','화요일','수요일','목요일','금요일','토요일','일요일']
weeks = pd.to_datetime(df['대여시간']).dt.dayofweek
df['weeks'] = weeks.apply(lambda x : days[x])
df

,순번,자전거고유번호,시작 대여소,대여시간,반납 대여소,반납시간,주행거리,주행시간,환승 유무,mile_bin,hour_bin,weeks
0,3672508,001_0110_02460,SJ_00439,2022-04-01 00:00:52,SJ_00222,2022-04-01 00:07:31,954,6,N,1KM,15분,금요일
1,3672509,001_0110_02587,SJ_00165,2022-04-01 00:01:17,SJ_00152,2022-04-01 00:11:25,1366,10,N,1.5KM,15분,금요일
2,3672510,001_0110_01483,SJ_00273,2022-04-01 00:01:39,SJ_00581,2022-04-01 01:25:33,14720,83,N,15KM,90분,금요일
3,3672511,001_0110_02910,SJ_00347,2022-04-01 00:01:58,SJ_00326,2022-04-01 00:11:25,1845,9,N,2KM,15분,금요일
4,3672512,001_0110_02059,SJ_00426,2022-04-01 00:02:03,SJ_00470,2022-04-01 00:12:26,1690,10,N,2KM,15분,금요일
...,...,...,...,...,...,...,...,...,...,...,...,...
1018924,4696249,001_0110_00901,SJ_00316,2022-07-31 23:56:57,SJ_00422,2022-08-01 00:02:39,648,5,N,0.7KM,15분,일요일
1018925,4696250,001_0110_00949,SJ_00240,2022-07-31 23:58:36,SJ_00347,2022-08-01 00:09:13,958,10,N,1KM,15분,일요일
1018926,4696251,001_0110_02458,SJ_00367,2022-07-31 23:59:09,SJ_00191,2022-08-01 00:05:40,1046,6,N,1.5KM,15분,일요일
1018927,4696252,001_0110_02154,SJ_00390,2022-07-31 23:59:25,SJ_00237,2022-08-01 00:11:53,1340,12,N,1.5KM,15분,일요일


In [43]:
df['dates'] = pd.to_datetime(df['대여시간']).dt.date
df['years'] = pd.to_datetime(df['대여시간']).dt.year
df['months'] = pd.to_datetime(df['대여시간']).dt.month
df['days'] = pd.to_datetime(df['대여시간']).dt.day
df['hours'] = pd.to_datetime(df['대여시간']).dt.hour
df['holiday'] = df['weeks'].apply(lambda x: '휴일' if x in ('일요일', '토요일') else '평일')
df['ymdh'] = df_imsi['대여시간'].str[:13]+':00'

# 공휴일
hoilday = ['2022-05-05','2022-05-08','2022-06-01','2022-06-06']
df['holiday'] = np.where(df['대여시간'].str[:10].isin(hoilday), '휴일', df['holiday'])

df.head(3)

,순번,자전거고유번호,시작 대여소,대여시간,반납 대여소,반납시간,주행거리,주행시간,환승 유무,mile_bin,hour_bin,weeks,dates,years,months,days,hours,holiday,ymdh
0,3672508,001_0110_02460,SJ_00439,2022-04-01 00:00:52,SJ_00222,2022-04-01 00:07:31,954,6,N,1KM,15분,금요일,2022-04-01,2022,4,1,0,평일,2022-04-01 00:00
1,3672509,001_0110_02587,SJ_00165,2022-04-01 00:01:17,SJ_00152,2022-04-01 00:11:25,1366,10,N,1.5KM,15분,금요일,2022-04-01,2022,4,1,0,평일,2022-04-01 00:00
2,3672510,001_0110_01483,SJ_00273,2022-04-01 00:01:39,SJ_00581,2022-04-01 01:25:33,14720,83,N,15KM,90분,금요일,2022-04-01,2022,4,1,0,평일,2022-04-01 00:00


In [44]:
# s=df.pivot_table(columns='holiday',index ='dates', values ='years', aggfunc='count',fill_value=0)
# s.to_csv('./ss.csv')

In [45]:
h1_ranges = [-1,7,9, 11, 13, 18, 20, 23]
h1_labels = ['심야', '출근', '오전', '점심', '오후','퇴근','야간']
df['hour_ditc'] = pd.cut(df['hours'], h1_ranges, labels = h1_labels)
df

,순번,자전거고유번호,시작 대여소,대여시간,반납 대여소,반납시간,주행거리,주행시간,환승 유무,mile_bin,hour_bin,weeks,dates,years,months,days,hours,holiday,ymdh,hour_ditc
0,3672508,001_0110_02460,SJ_00439,2022-04-01 00:00:52,SJ_00222,2022-04-01 00:07:31,954,6,N,1KM,15분,금요일,2022-04-01,2022,4,1,0,평일,2022-04-01 00:00,심야
1,3672509,001_0110_02587,SJ_00165,2022-04-01 00:01:17,SJ_00152,2022-04-01 00:11:25,1366,10,N,1.5KM,15분,금요일,2022-04-01,2022,4,1,0,평일,2022-04-01 00:00,심야
2,3672510,001_0110_01483,SJ_00273,2022-04-01 00:01:39,SJ_00581,2022-04-01 01:25:33,14720,83,N,15KM,90분,금요일,2022-04-01,2022,4,1,0,평일,2022-04-01 00:00,심야
3,3672511,001_0110_02910,SJ_00347,2022-04-01 00:01:58,SJ_00326,2022-04-01 00:11:25,1845,9,N,2KM,15분,금요일,2022-04-01,2022,4,1,0,평일,2022-04-01 00:00,심야
4,3672512,001_0110_02059,SJ_00426,2022-04-01 00:02:03,SJ_00470,2022-04-01 00:12:26,1690,10,N,2KM,15분,금요일,2022-04-01,2022,4,1,0,평일,2022-04-01 00:00,심야
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018924,4696249,001_0110_00901,SJ_00316,2022-07-31 23:56:57,SJ_00422,2022-08-01 00:02:39,648,5,N,0.7KM,15분,일요일,2022-07-31,2022,7,31,23,휴일,2022-07-31 23:00,야간
1018925,4696250,001_0110_00949,SJ_00240,2022-07-31 23:58:36,SJ_00347,2022-08-01 00:09:13,958,10,N,1KM,15분,일요일,2022-07-31,2022,7,31,23,휴일,2022-07-31 23:00,야간
1018926,4696251,001_0110_02458,SJ_00367,2022-07-31 23:59:09,SJ_00191,2022-08-01 00:05:40,1046,6,N,1.5KM,15분,일요일,2022-07-31,2022,7,31,23,휴일,2022-07-31 23:00,야간
1018927,4696252,001_0110_02154,SJ_00390,2022-07-31 23:59:25,SJ_00237,2022-08-01 00:11:53,1340,12,N,1.5KM,15분,일요일,2022-07-31,2022,7,31,23,휴일,2022-07-31 23:00,야간


In [46]:
df.groupby(['hour_ditc','hours'])['hours'].count()

hour_ditc  hours
심야         0        15340
           1         6853
           2         3395
           3         1805
           4         1594
                    ...  
야간         19           0
           20           0
           21       57121
           22       43669
           23       26656
Name: hours, Length: 168, dtype: int64

#### 대여장소, 반납장소 , 결측치 정비

In [140]:
# 대여장소
df_imsi = pd.merge(df, df_spot, left_on ='시작 대여소', right_on ='대여소 아이디', how='left')

df_imsi = df_imsi[['순번', '자전거고유번호', '시작 대여소', '대여시간', '반납 대여소', '반납시간', '주행거리', '주행시간',
       '환승 유무', 'mile_bin', 'hour_bin', 'weeks', 'years','months','days','hours', 'holiday', 'ymdh', 'hour_ditc','주소(동)']]

df_imsi.rename(columns={'주소(동)':'st_spot'},inplace=True)

In [141]:
# 반납장소
df_imsi = pd.merge(df_imsi, df_spot, left_on ='반납 대여소', right_on ='대여소 아이디', how='left')

df_imsi = df_imsi[['순번', '자전거고유번호', '시작 대여소', '대여시간', '반납 대여소', '반납시간', '주행거리', '주행시간',
       '환승 유무', 'mile_bin', 'hour_bin', 'weeks', 'years','months','days','hours','holiday', 'ymdh', 'hour_ditc','st_spot','주소(동)']]

df_imsi.rename(columns={'주소(동)':'ed_spot'},inplace=True)

In [142]:
# 첫번째 결측값 정비
df_imsi['st_spot'].fillna(df_imsi['ed_spot'], inplace=True)
df_imsi['ed_spot'].fillna(df_imsi['st_spot'], inplace=True)

# 두번째 결측값 정비
df_imsi['st_spot'].fillna(df_imsi['시작 대여소'], inplace=True)
df_imsi['ed_spot'].fillna(df_imsi['반납 대여소'], inplace=True)


In [143]:
df_imsi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1018929 entries, 0 to 1018928
Data columns (total 21 columns):
 #   Column     Non-Null Count    Dtype   
---  ------     --------------    -----   
 0   순번         1018929 non-null  int64   
 1   자전거고유번호    1018929 non-null  object  
 2   시작 대여소     1018929 non-null  object  
 3   대여시간       1018929 non-null  object  
 4   반납 대여소     1018929 non-null  object  
 5   반납시간       1018929 non-null  object  
 6   주행거리       1018929 non-null  int64   
 7   주행시간       1018929 non-null  int64   
 8   환승 유무      1018929 non-null  object  
 9   mile_bin   1018929 non-null  category
 10  hour_bin   1018929 non-null  category
 11  weeks      1018929 non-null  object  
 12  years      1018929 non-null  int64   
 13  months     1018929 non-null  int64   
 14  days       1018929 non-null  int64   
 15  hours      1018929 non-null  int64   
 16  holiday    1018929 non-null  object  
 17  ymdh       1018929 non-null  object  
 18  hour_ditc  1018929 non

In [61]:
# 결측행 조회
df_imsi[df_imsi.isnull().any(axis=1)]

,순번,자전거고유번호,시작 대여소,대여시간,반납 대여소,반납시간,주행거리,주행시간,환승 유무,mile_bin,...,weeks,years,months,days,hours,holiday,ymdh,hour_ditc,st_spot,ed_spot


#### 날씨정보(시간대) 수집 (기상청 기상자료개방포털) 및 통합
##### 수집사이트 주소 : https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36
##### ljk7312@hanmail  java6***!

In [144]:
weather = pd.read_csv('./세종지역_시간대별날씨정보.csvv',encoding='euc-kr')
weather = weather.fillna(0)
weather

,지점,지점명,일시,기온(°C),강수량(mm),풍속(m/s),일조(hr),전운량(10분위),지면온도(°C)
0,239,세종,2022-04-01 00:00,8.5,0.0,2.9,0.0,9.0,6.1
1,239,세종,2022-04-01 01:00,7.7,0.0,2.7,0.0,10.0,5.3
2,239,세종,2022-04-01 02:00,6.7,0.0,2.9,0.0,10.0,4.6
3,239,세종,2022-04-01 03:00,6.2,0.0,1.8,0.0,6.0,3.5
4,239,세종,2022-04-01 04:00,5.7,0.0,1.2,0.0,6.0,2.8
...,...,...,...,...,...,...,...,...,...
2923,239,세종,2022-07-31 19:00,25.2,4.3,1.9,0.0,10.0,26.3
2924,239,세종,2022-07-31 20:00,25.2,0.6,1.5,0.0,10.0,26.3
2925,239,세종,2022-07-31 21:00,25.2,0.0,1.2,0.0,8.0,26.2
2926,239,세종,2022-07-31 22:00,25.1,0.0,0.3,0.0,10.0,26.2


In [145]:
# 날씨 데이터 통합
df_imsi = pd.merge(df_imsi, weather, left_on ='ymdh', right_on ='일시', how = 'left')

df_imsi = df_imsi[['순번', '자전거고유번호', '시작 대여소', '대여시간', '반납 대여소', '반납시간', '주행거리', '주행시간',
                   '환승 유무', 'mile_bin', 'hour_bin', 'weeks', 'years', 'months', 'days','hours', 'holiday', 'ymdh', 'hour_ditc', 'st_spot', 'ed_spot',
                   '기온(°C)', '강수량(mm)', '풍속(m/s)', '일조(hr)','전운량(10분위)','지면온도(°C)']]

df_imsi.rename(columns={'기온(°C)':'temp','강수량(mm)':'rain','풍속(m/s)':'wind','일조(hr)':'sunshine','전운량(10분위)':'cloud','지면온도(°C)':'ground_temp'},inplace=True)

df_imsi

,순번,자전거고유번호,시작 대여소,대여시간,반납 대여소,반납시간,주행거리,주행시간,환승 유무,mile_bin,...,ymdh,hour_ditc,st_spot,ed_spot,temp,rain,wind,sunshine,cloud,ground_temp
0,3672508,001_0110_02460,SJ_00439,2022-04-01 00:00:52,SJ_00222,2022-04-01 00:07:31,954,6,N,1KM,...,2022-04-01 00:00,심야,소담동,보람동,8.5,0.0,2.9,0.0,9.0,6.1
1,3672509,001_0110_02587,SJ_00165,2022-04-01 00:01:17,SJ_00152,2022-04-01 00:11:25,1366,10,N,1.5KM,...,2022-04-01 00:00,심야,아름동,종촌동,8.5,0.0,2.9,0.0,9.0,6.1
2,3672510,001_0110_01483,SJ_00273,2022-04-01 00:01:39,SJ_00581,2022-04-01 01:25:33,14720,83,N,15KM,...,2022-04-01 00:00,심야,나성동,나성동,8.5,0.0,2.9,0.0,9.0,6.1
3,3672511,001_0110_02910,SJ_00347,2022-04-01 00:01:58,SJ_00326,2022-04-01 00:11:25,1845,9,N,2KM,...,2022-04-01 00:00,심야,나성동,어진동,8.5,0.0,2.9,0.0,9.0,6.1
4,3672512,001_0110_02059,SJ_00426,2022-04-01 00:02:03,SJ_00470,2022-04-01 00:12:26,1690,10,N,2KM,...,2022-04-01 00:00,심야,조치원읍 죽림리,조치원읍 서창리,8.5,0.0,2.9,0.0,9.0,6.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018924,4696249,001_0110_00901,SJ_00316,2022-07-31 23:56:57,SJ_00422,2022-08-01 00:02:39,648,5,N,0.7KM,...,2022-07-31 23:00,야간,나성동,다정동,25.3,0.0,1.2,0.0,9.0,26.1
1018925,4696250,001_0110_00949,SJ_00240,2022-07-31 23:58:36,SJ_00347,2022-08-01 00:09:13,958,10,N,1KM,...,2022-07-31 23:00,야간,다정동,나성동,25.3,0.0,1.2,0.0,9.0,26.1
1018926,4696251,001_0110_02458,SJ_00367,2022-07-31 23:59:09,SJ_00191,2022-08-01 00:05:40,1046,6,N,1.5KM,...,2022-07-31 23:00,야간,아름동,도담동,25.3,0.0,1.2,0.0,9.0,26.1
1018927,4696252,001_0110_02154,SJ_00390,2022-07-31 23:59:25,SJ_00237,2022-08-01 00:11:53,1340,12,N,1.5KM,...,2022-07-31 23:00,야간,나성동,다정동,25.3,0.0,1.2,0.0,9.0,26.1


In [157]:
# 강수 여부
df_imsi['rain_yn'] = df_imsi['rain'].apply(lambda x : '비옴' if x >0 else '비안옴')

# 바람세기
w_ranges = [-1, 0.2, 1.5, 3.3, 5.4, 7, 10]
w_labels = ['고요', '미풍', '남실바람', '산들바람', '건들바람','흔들바람']
df_imsi['wind_bins'] = pd.cut(df_imsi['wind'], w_ranges, labels = w_labels)
df_imsi['wind_bins']

# 구름 세기
c_ranges = [-1, 2, 5, 7, 10]
c_labels = ['맑음', '구름조금', '구름많음', '흐림']
df_imsi['cloud_bins'] = pd.cut(df_imsi['cloud'], c_ranges, labels = c_labels)
df_imsi['cloud_bins']

# 온도 세기
t_ranges = [-20, 0, 10, 20, 30, 40]
t_labels = ['추움', '쌀쌀', '쾌적', '따뜻', '더움']
df_imsi['temp_bins'] = pd.cut(df_imsi['temp'], t_ranges, labels = t_labels)
df_imsi['temp_bins']

# df_imsi[df_imsi.isnull().any(axis=1)]

0          쌀쌀
1          쌀쌀
2          쌀쌀
3          쌀쌀
4          쌀쌀
           ..
1018924    따뜻
1018925    따뜻
1018926    따뜻
1018927    따뜻
1018928    따뜻
Name: temp_bins, Length: 1018929, dtype: category
Categories (5, object): ['추움' < '쌀쌀' < '쾌적' < '따뜻' < '더움']

In [158]:
df_imsi

,순번,자전거고유번호,시작 대여소,대여시간,반납 대여소,반납시간,주행거리,주행시간,환승 유무,mile_bin,...,temp,rain,wind,sunshine,cloud,ground_temp,rain_yn,wind_bins,cloud_bins,temp_bins
0,3672508,001_0110_02460,SJ_00439,2022-04-01 00:00:52,SJ_00222,2022-04-01 00:07:31,954,6,N,1KM,...,8.5,0.0,2.9,0.0,9.0,6.1,비안옴,남실바람,흐림,쌀쌀
1,3672509,001_0110_02587,SJ_00165,2022-04-01 00:01:17,SJ_00152,2022-04-01 00:11:25,1366,10,N,1.5KM,...,8.5,0.0,2.9,0.0,9.0,6.1,비안옴,남실바람,흐림,쌀쌀
2,3672510,001_0110_01483,SJ_00273,2022-04-01 00:01:39,SJ_00581,2022-04-01 01:25:33,14720,83,N,15KM,...,8.5,0.0,2.9,0.0,9.0,6.1,비안옴,남실바람,흐림,쌀쌀
3,3672511,001_0110_02910,SJ_00347,2022-04-01 00:01:58,SJ_00326,2022-04-01 00:11:25,1845,9,N,2KM,...,8.5,0.0,2.9,0.0,9.0,6.1,비안옴,남실바람,흐림,쌀쌀
4,3672512,001_0110_02059,SJ_00426,2022-04-01 00:02:03,SJ_00470,2022-04-01 00:12:26,1690,10,N,2KM,...,8.5,0.0,2.9,0.0,9.0,6.1,비안옴,남실바람,흐림,쌀쌀
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018924,4696249,001_0110_00901,SJ_00316,2022-07-31 23:56:57,SJ_00422,2022-08-01 00:02:39,648,5,N,0.7KM,...,25.3,0.0,1.2,0.0,9.0,26.1,비안옴,미풍,흐림,따뜻
1018925,4696250,001_0110_00949,SJ_00240,2022-07-31 23:58:36,SJ_00347,2022-08-01 00:09:13,958,10,N,1KM,...,25.3,0.0,1.2,0.0,9.0,26.1,비안옴,미풍,흐림,따뜻
1018926,4696251,001_0110_02458,SJ_00367,2022-07-31 23:59:09,SJ_00191,2022-08-01 00:05:40,1046,6,N,1.5KM,...,25.3,0.0,1.2,0.0,9.0,26.1,비안옴,미풍,흐림,따뜻
1018927,4696252,001_0110_02154,SJ_00390,2022-07-31 23:59:25,SJ_00237,2022-08-01 00:11:53,1340,12,N,1.5KM,...,25.3,0.0,1.2,0.0,9.0,26.1,비안옴,미풍,흐림,따뜻
